In [1]:
from Model.NQModel import NQModel
from Model.LossFn import LossFn
import torch
import time
import sklearn
import Model.datasetutils as datasetutils
import Model.tensorboardutils as boardutils
import torch.utils.tensorboard as tensorboard
from tqdm import tqdm_notebook as tqdm
import transformers
import matplotlib.pyplot as plt

## Constants

In [2]:
TensorBoardLocation = 'runs/NQ_TIME:{}'.format(int((time.time() - 1583988084)/60))
TensorBoardLocation

'runs/NQ_TIME:10159'

In [3]:
epochs = 1 # no loop 
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
writer = tensorboard.SummaryWriter(TensorBoardLocation)

## Dataset

In [4]:
traingen, validgen = datasetutils.get_dataset(device)

## MODEL

In [5]:
model = NQModel()
#writer.add_graph(model, (trainData[0][0], trainData[0][2], trainData[0][1]))

In [6]:
optim = transformers.AdamW(model.parameters())
scheduler = transformers.get_cosine_schedule_with_warmup(optim, num_warmup_steps=100, num_training_steps=800,num_cycles=0.5, last_epoch=-1)

## Confusion Matrix

In [7]:
AnswerTypes = ['Wrong Ans', 'Short Ans', 'Yes No']
YesNoLabels = ['No', 'Yes']

In [8]:
def update_confusion_matrix(ATMatrix, YNMatrix, StartM, EndM, output, target) : 
    predsT = output[0].argmax(dim = 1)
    truthT = target[0].argmax(dim = 1)

    for x, y in zip(predsT, truthT) : 
        ATMatrix[x][y] += 1


    predsYN = (torch.sigmoid(output[3].flatten()) >= 0.5) + 1 -1
    truthYN = target[3].flatten()

    for x, y in zip(predsYN, truthYN) : 
        YNMatrix[x][y] += 1    


    start01 = (torch.sigmoid(output[1].flatten()) >= 0.5) +1 -1
    end01   = (torch.sigmoid(output[2].flatten()) >= 0.5) +1 -1

    startcm = sklearn.metrics.confusion_matrix(target[1].flatten().detach().numpy(), start01)
    endcm   = sklearn.metrics.confusion_matrix(target[2].flatten().detach().numpy(), end01)

    StartM += torch.from_numpy(startcm)
    EndM   += torch.from_numpy(endcm)    

In [9]:
def log_confusion_matrix(matrix, labels, name, step): 
    opfigure = boardutils.confusion_matrix_image(matrix.detach().numpy(), labels)
    writer.add_figure(name, opfigure, step)
    
def log_matrices(AnsTypeM, YNM, StM, EndM, call_type, steps):
    log_confusion_matrix(AnsTypeM, AnswerTypes, "Answer type confusion matrix" + call_type, steps)
    log_confusion_matrix(YNM, YesNoLabels, "Yes No confusion matrix" + call_type, steps)
    log_confusion_matrix(StM, YesNoLabels, "Start confusion matrix" + call_type, steps)
    log_confusion_matrix(EndM, YesNoLabels, "End confusion matrix" + call_type, steps) 

## Run

In [10]:
def validate(val_num) : 
    model.eval()
    
    ValAnswerTypeMatrix = torch.zeros([3, 3], requires_grad = False)
    ValYesNoMatrix      = torch.zeros([2, 2], requires_grad = False)
    ValStartMatrix      = torch.zeros([2, 2], requires_grad = False)
    ValEndMatrix        = torch.zeros([2, 2], requires_grad = False)
    
    with torch.no_grad():
        for inp_id, inp_type, inp_mask, ans_type, start, end, yes_no in tqdm(validgen) : 
            output = model(inp_id.squeeze(), inp_mask.squeeze(), inp_type.squeeze())  

            update_confusion_matrix(ValAnswerTypeMatrix, ValYesNoMatrix, ValStartMatrix, ValEndMatrix, output, (ans_type, start, end, yes_no))

            ## Calculate Loss
            AT_loss = LossFn.loss_AT(output[0], ans_type.squeeze().argmax(1))
            Start_loss = LossFn.loss_start(output[1], start.squeeze().type(torch.FloatTensor))
            End_loss = LossFn.loss_end(output[2], end.squeeze().type(torch.FloatTensor))
            Yes_no_loss = LossFn.loss_yes_no(output[3], yes_no.squeeze())

            ## Save loss values
            writer.add_scalars('Loss values Validation',
                {"AT_loss_val" : AT_loss,"Start_loss_val":Start_loss, "End_loss_val":End_loss, "Yes_no_loss_val":Yes_no_loss},
                val_num, time.time())


    log_matrices(ValAnswerTypeMatrix, ValYesNoMatrix, ValStartMatrix, ValEndMatrix, " eval", val_num)    

In [11]:
def train() : 
    AnswerTypeMatrix = torch.zeros([3,3], requires_grad = False)
    YesNoMatrix      = torch.zeros([2,2], requires_grad = False)
    StartMatrix      = torch.zeros([2,2], requires_grad = False)
    EndMatrix        = torch.zeros([2,2], requires_grad = False)

    model.train()
    steps = -1

    for inp_id, inp_type, inp_mask, ans_type, start, end, yes_no in tqdm(traingen) : 
        steps += 1
        output = model(inp_id.squeeze(), inp_mask.squeeze(), inp_type.squeeze())

        ## Calculate Confusion Matrix
        update_confusion_matrix(AnswerTypeMatrix, YesNoMatrix, StartMatrix, EndMatrix,output, (ans_type, start, end, yes_no))
        if steps%5 == 0 : log_matrices(AnswerTypeMatrix, YesNoMatrix, StartMatrix, EndMatrix, " train", steps)

        ## Calculate Loss
        AT_loss = LossFn.loss_AT(output[0], ans_type.squeeze().argmax(1))
        Start_loss = LossFn.loss_start(output[1], start.squeeze().type(torch.FloatTensor))
        End_loss = LossFn.loss_end(output[2], end.squeeze().type(torch.FloatTensor))
        Yes_no_loss = LossFn.loss_yes_no(output[3], yes_no.squeeze())
        
        ## Update model params and optim/sched
        total_loss = AT_loss + Start_loss + End_loss + Yes_no_loss
        total_loss.backward()

        ## Save loss values
        writer.add_scalars('Loss values',
            {"AT_loss" : AT_loss.item(),"Start_loss":Start_loss.item(), "End_loss":End_loss.item(), "Yes_no_loss":Yes_no_loss.item()},
            steps, time.time())


        optim.step()
        scheduler.step()     
        
        if steps%20 == 0 : validate(steps/20)

In [12]:
train()

/Users/abhash.ya/Desktop/Competitive DS/QA/Model/tensorboardutils.py:22: RuntimeWarning: invalid value encountered in true_divide
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=3)


Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Pyth

KeyboardInterrupt: 